Installing Necessary Packages

In [ ]:
!pip install -r requirements.txt

User Inputs

In [2]:
# User Inputs #
Gemini_API_Keys = [
    "YOUR API 1",
    "YOUR API 2"
    ]
newspaper_name = "UNB"
date = "10-01-2025"
API = Gemini_API_Keys[-1] # Choose any
url = "https://unb.com.bd/category/14/Bangladesh"
DEBUG = True



Pipeline codes and outputs

In [ ]:
# Pipeline Codes #
print("---------------------------------------------------")
print(f"Starting test for Newspaper: **   {newspaper_name}   **")
print("---------------------------------------------------")

# Function to check if a newspaper name exists in a file and return its type
def check_newspaper_type(newspaper_name):
    pagination_file = "pagination_website_lists.txt"
    dynamic_file = "dynamic_website_lists.txt"

    # Check in pagination file
    try:
        with open(pagination_file, "r", encoding="utf-8") as file:
            if newspaper_name in file.read().splitlines():
                return "The website is of type pagination."
    except FileNotFoundError:
        pass

    # Check in dynamic file
    try:
        with open(dynamic_file, "r", encoding="utf-8") as file:
            if newspaper_name in file.read().splitlines():
                return "The website is of type dynamic."
    except FileNotFoundError:
        pass

    return "Error-404: The newspaper name is not found in either list. Generate Scraping Code First."


# Check the type of the newspaper
result = check_newspaper_type(newspaper_name)

# Print the result
print(result)

infinite = False
if "Error-404" in result:
    # Webpage Classification #
    import webpage_classifier_agent
    webpage_type = webpage_classifier_agent.classifier(API, url).split('<type>')[1]
    print(webpage_type)

    print(f"Starting scraping code generation for newspaper: {newspaper_name}")
    if 'pagination' in webpage_type:
        import pagination_code_writer_agent
        pagination_code_writer_agent.paginate_code_writer(url, newspaper_name, API)
        # Append the newspaper name to the appropriate file
        file_name = "pagination_website_lists.txt"
        with open(file_name, "a", encoding="utf-8") as file:
            file.write(newspaper_name + "\n")
    elif 'dynamic' in webpage_type:
        import dynamic_code_writer_agent
        dynamic_code_writer_agent.dynamic_code_writer(API, url, newspaper_name)
        file_name = "dynamic_website_lists.txt"
        with open(file_name, "a", encoding="utf-8") as file:
            file.write(newspaper_name + "\n")
    elif 'Infinite' in webpage_type:
        result = "The website is of type Infinite Scroll"
        infinite = True
    else:
        print("Error: Could not classify the webpage.")
    if infinite == False:
        # Check the type of the newspaper after code generation
        result = check_newspaper_type(newspaper_name)

print("result variable: ", result)
from datetime import datetime
def days_between_today_and(date_str):
    """
    Calculate the number of days between today and the given date.

    Parameters:
        date_str (str): The target date as a string in the format 'YYYY-MM-DD'.

    Returns:
        int: Number of days between today and the given date.
            Positive if the date is in the future, negative if in the past.
    """
    try:
        # Parse the input date string
        target_date = datetime.strptime(date_str, '%d-%m-%Y').date()
        
        # Get today's date
        today_date = datetime.today().date()
        
        # Calculate the difference in days
        delta = (target_date - today_date).days
        
        return delta
    except ValueError:
        return "Invalid date format. Please use 'DD-MM-YYYY'."
    
print(f"-------------------  Main Output for newspaper:     {newspaper_name}   -------------------------")

if 'pagination' in result:
    no_of_pages_to_scrape = (abs(days_between_today_and(date)) + 1) * 3
    ## Importing Button Click Function ##
    import importlib.util

    # Specify the module's path
    module_path = f"{newspaper_name}/button_click.py"
    module_name = "button_click"

    # Load the module
    spec = importlib.util.spec_from_file_location(module_name, module_path)
    module = importlib.util.module_from_spec(spec)
    spec.loader.exec_module(module)

    url_list = module.pagination_url_collector(no_of_pages_to_scrape)

    import pagination_title_link_collector
    news_df = pagination_title_link_collector.collect(API, url_list)
    news_df.drop_duplicates(inplace=True)
    news_df.reset_index(drop=True, inplace=True)
    print("Total number of news to be processed: ",len(news_df))
    if DEBUG == True:
        news_df = news_df.head(10)
    len(news_df)

    import report_processor_agent
    final_df = report_processor_agent.report_processor(news_df, Gemini_API_Keys)
elif 'dynamic' in result:
    no_of_button_click = (abs(days_between_today_and(date)) + 1) * 2
    # import universal_dynamic_title_link
    # news_df = universal_dynamic_title_link.title_link_dict_func(API, url, newspaper_name, no_of_button_click)

    import universal_dynamic_title_link_div
    news_df = universal_dynamic_title_link_div.title_link_dict_func(API, url, newspaper_name, no_of_button_click)

    print("Total number of news to be processed: ",len(news_df))
    if DEBUG == True:
        news_df = news_df.head(10)
    import report_processor_agent
    final_df = report_processor_agent.report_processor(news_df, Gemini_API_Keys)
elif 'Infinite' in result:
    import infinite_scroll_title_link
    no_of_button_click = (abs(days_between_today_and(date)) + 1) * 2
    news_df = infinite_scroll_title_link.title_link_dict_func(API, url, newspaper_name, no_of_button_click)
    print("Total number of news to be processed: ",len(news_df))
    if DEBUG == True:
        news_df = news_df.head(10)
    import report_processor_agent
    final_df = report_processor_agent.report_processor(news_df, Gemini_API_Keys)
else:
    print("ERROR: Could not identify website type")

print(final_df.head(10))

